In [1]:
import requests
import mwclient
import json, os
import csv
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta

In [2]:
!pip install duckdb

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.3 MB 5.6 MB/s eta 0:00:03
   ----------- ---------------------------- 3.7/12.3 MB 9.9 MB/s eta 0:00:01
   --------------------- ------------------ 6.6/12.3 MB 11.5 MB/s eta 0:00:01
   ----------------------------- ---------- 9.2/12.3 MB 11.9 MB/s eta 0:00:01
   -------------------------------------- - 11.8/12.3 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 10.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\grace\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
import duckdb

# Placeholder for the database connection. It will be initialized later with the URL.
conn = duckdb.connect()
conn
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

In [25]:
database_url = "https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb"

# Attach the remote file as a database named 'web_db' and start using it
try:
    conn.execute(f"ATTACH '{database_url}' AS web_db (READ_ONLY);")
    conn.execute("USE web_db;")
    print(f"Successfully attached database from: {database_url}")
except Exception as e:
    print(f"Error attaching database: {e}")

Successfully attached database from: https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb


In [26]:
query = "PRAGMA show_tables"
result = conn.sql(query)
result

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ data_table │
└────────────┘

In [27]:
table_name = "data_table"
query = f"PRAGMA table_info('web_db.{table_name}');"

# We can apply the method .df() to the result of the query to convert it into a dataframe
column_info_df = conn.sql(query).df()
column_info_df

,cid,name,type,notnull,dflt_value,pk
0,0,date,DATE,False,None,False
1,1,country,VARCHAR,False,None,False
2,2,country_code,VARCHAR,False,None,False
3,3,project,VARCHAR,False,None,False
4,4,page_id,BIGINT,False,None,False
5,5,article,VARCHAR,False,None,False
6,6,qid,VARCHAR,False,None,False
7,7,pageviews,BIGINT,False,None,False


In [31]:
query_2 = """
SELECT article, country, pageviews
FROM data_table
LIMIT 5;
"""
result_2 = conn.sql(query_2).df()

result_2

,article,country,pageviews
0,Ravi_Shankar,United States of America,966
1,John_Galt,United States of America,451
2,Carlos_Tevez,United States of America,529
3,Ruth_Buzzi,United States of America,838
4,Robert_Smith_(running_back),United States of America,2347


In [14]:
url = "https://api.wp1.openzim.org/v1/projects/Physics/articles?page=1&numRows=100"
physics_table = requests.get(url)
articles = physics_table.json()['articles']

In [5]:
import requests
import pandas as pd

def fetch_all_physics_articles(num_rows=200):
    page = 1
    all_articles = []

    while True:
        url = f"https://api.wp1.openzim.org/v1/projects/Physics/articles?page={page}&numRows={num_rows}"
        r = requests.get(url)

        if r.status_code != 200:
            print("Error:", r.status_code)
            break

        data = r.json()
        articles = data.get("articles", [])

        if not articles:  # stop when no more pages
            break

        all_articles.extend(articles)
        page += 1

    return pd.DataFrame(all_articles)

physics_df = fetch_all_physics_articles()
physics_df.head()

physics_titles = (
    physics_df["article"]
    .str.replace(" ", "_")
    .tolist()
)


In [6]:
len(physics_titles)

28391

In [7]:
conn.sql("SELECT count(*) FROM data_table;").df()


,count_star()
0,49290960


In [8]:
conn.sql("PRAGMA table_info('data_table');").df()


,cid,name,type,notnull,dflt_value,pk
0,0,date,DATE,False,None,False
1,1,country,VARCHAR,False,None,False
2,2,country_code,VARCHAR,False,None,False
3,3,project,VARCHAR,False,None,False
4,4,page_id,BIGINT,False,None,False
5,5,article,VARCHAR,False,None,False
6,6,qid,VARCHAR,False,None,False
7,7,pageviews,BIGINT,False,None,False


In [10]:
physics_articles_df = pd.DataFrame({"article": physics_titles})
conn.register("physics_articles", physics_articles_df)

physics_pageviews = conn.sql(
    """
    SELECT d.*
    FROM data_table d
    JOIN physics_articles p
    ON d.article = p.article
    """
).df()


In [13]:
physics_pageviews.date.unique()

<DatetimeArray>
['2024-01-08 00:00:00', '2024-01-09 00:00:00', '2024-01-29 00:00:00',
 '2024-01-30 00:00:00', '2024-02-09 00:00:00', '2024-02-10 00:00:00',
 '2024-02-28 00:00:00', '2024-03-04 00:00:00', '2024-03-21 00:00:00',
 '2024-03-22 00:00:00',
 ...
 '2024-03-18 00:00:00', '2024-04-13 00:00:00', '2024-07-20 00:00:00',
 '2024-09-15 00:00:00', '2024-12-25 00:00:00', '2024-01-22 00:00:00',
 '2024-06-30 00:00:00', '2024-11-23 00:00:00', '2024-12-28 00:00:00',
 '2024-12-29 00:00:00']
Length: 366, dtype: datetime64[us]

In [14]:
physics_pageviews.to_csv("physics_pageviews_2024.csv", index=False)


Doing same for 2023 DuckDB, excluding data before feb 06

In [28]:
conn = duckdb.connect()
conn
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

database_url = "https://cs.wellesley.edu/~eni/duckdb/2023_wiki_views.duckdb"

# Attach the remote file as a database named 'web_db' and start using it
try:
    conn.execute(f"ATTACH '{database_url}' AS web_db (READ_ONLY);")
    conn.execute("USE web_db;")
    print(f"Successfully attached database from: {database_url}")
except Exception as e:
    print(f"Error attaching database: {e}")

Successfully attached database from: https://cs.wellesley.edu/~eni/duckdb/2023_wiki_views.duckdb


In [29]:
query = "PRAGMA show_tables"
result = conn.sql(query)
result

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ data_table │
└────────────┘

In [32]:
table_name = "data_table"
query = f"PRAGMA table_info('web_db.{table_name}');"

# We can apply the method .df() to the result of the query to convert it into a dataframe
column_info_df = conn.sql(query).df()
column_info_df

query_2 = """
SELECT article, country, pageviews
FROM data_table
LIMIT 5;
"""
result_2 = conn.sql(query_2).df()

result_2

,article,country,pageviews
0,Ravi_Shankar,United States of America,966
1,John_Galt,United States of America,451
2,Carlos_Tevez,United States of America,529
3,Ruth_Buzzi,United States of America,838
4,Robert_Smith_(running_back),United States of America,2347


In [33]:
conn.sql("SELECT count(*) FROM data_table;").df()

,count_star()
0,23381823


In [35]:
conn.register("physics_articles", physics_articles_df)

physics_pageviews_2023 = conn.sql(
    """
    SELECT d.*
    FROM data_table d
    JOIN physics_articles p
    ON d.article = p.article
    WHERE d.date >= DATE '2023-02-06'
    """
).df()

In [38]:
physics_pageviews_2023.to_csv("physics_pageviews_2023.csv", index=False)

In [39]:
df23 = pd.read_csv("physics_pageviews_2023.csv")
df24 = pd.read_csv("physics_pageviews_2024.csv")

merged = pd.concat([df23, df24], ignore_index=True)
merged.to_csv("physics_pageviews.csv", index=False)

In [42]:
agg_daily = merged.groupby("date", as_index=False)["pageviews"].sum()
agg_daily.to_csv("daily_views.csv", index=False)
len(agg_daily)

695

In [43]:
agg_article = merged.groupby("article", as_index=False)["pageviews"].sum()
agg_article.to_csv("article_views.csv", index=False)
len(agg_article)

4762